In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense
from livelossplot import PlotLossesKeras
from keras.regularizers import l2 # L2-regularisation

In [ ]:
height, width = 252, 252
batch_size = 100
datagen = ImageDataGenerator(rescale=1./ 255, samplewise_center=True, samplewise_std_normalization=True, validation_split=0.1)
train_generator = datagen.flow_from_directory(
        '/home/user/Steganalysis/master-diploma/grayscale_databases/Dryuchenko-filtered',
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        color_mode='grayscale',
        target_size = (height, width))

validation_generator = datagen.flow_from_directory(
        '/home/user/Steganalysis/master-diploma/grayscale_databases/Dryuchenko-filtered',
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        color_mode='grayscale',
        target_size = (height, width))

In [ ]:
l2_lambda = 0.0001; #use 0.0001 as a L2-regularisation factor
model = Sequential()
model.add(Conv2D(1, 3, activation='tanh', input_shape=(height,width,1), W_regularizer=l2(l2_lambda),init='he_normal'))
model.add(Conv2D(64, 249, activation='tanh', W_regularizer=l2(l2_lambda),init='he_normal'))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
sgd = keras.optimizers.SGD(lr=0.005, momentum=0.0, decay=0.0000005, nesterov=False)
model.compile(sgd, 'categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=30,
        callbacks=[PlotLossesKeras()],
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size,
        verbose=1)